<a href="https://colab.research.google.com/github/michiWS1920/nfl_data/blob/master/playingaround.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
library(tidyverse)
library(ggplot2)


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.1     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [ ]:
games_data <- read_csv(url("https://github.com/ryurko/nflscrapR-data/raw/master/roster_data/regular_season/reg_roster_2019.csv"))

In [2]:
draft_data <- read_csv(url("https://raw.githubusercontent.com/leesharpe/nfldata/master/data/draft_picks.csv"))

Parsed with column specification:
cols(
  season = col_double(),
  team = col_character(),
  round = col_double(),
  pick = col_double(),
  playerid = col_character(),
  full_name = col_character(),
  name = col_character(),
  side = col_character(),
  category = col_character(),
  position = col_character()
)



In [3]:
draft_data

season,team,round,pick,playerid,full_name,name,side,category,position
<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2000,CLE,1,1,BrowCo22,Courtney Brown,C.Brown,D,DL,DE
2000,WAS,1,2,ArriLa00,LaVar Arrington,L.Arrington,D,LB,LB
2000,WAS,1,3,SamuCh00,Chris Samuels,C.Samuels,O,OL,T
2000,CIN,1,4,WarrPe00,Peter Warrick,P.Warrick,O,WR,WR
2000,BAL,1,5,LewiJa00,Jamal Lewis,J.Lewis,O,RB,RB
2000,PHI,1,6,SimoCo00,Corey Simon,C.Simon,D,DL,DT
2000,ARI,1,7,JoneTh00,Thomas Jones,T.Jones,O,RB,RB
2000,PIT,1,8,BurrPl00,Plaxico Burress,P.Burress,O,WR,WR
2000,CHI,1,9,UrlaBr00,Brian Urlacher,B.Urlacher,D,LB,LB


In [6]:
games_data <- read_csv(url("https://raw.githubusercontent.com/ryurko/nflscrapR-data/master/play_by_play_data/regular_season/reg_pbp_2019.csv"))

Parsed with column specification:
cols(
  .default = col_double(),
  home_team = col_character(),
  away_team = col_character(),
  posteam = col_character(),
  posteam_type = col_character(),
  defteam = col_character(),
  side_of_field = col_character(),
  game_date = col_date(format = ""),
  game_half = col_character(),
  time = col_time(format = ""),
  yrdln = col_character(),
  desc = col_character(),
  play_type = col_character(),
  pass_length = col_character(),
  pass_location = col_character(),
  run_location = col_character(),
  run_gap = col_character(),
  field_goal_result = col_character(),
  extra_point_result = col_character(),
  two_point_conv_result = col_character(),
  timeout_team = col_character()
  # ... with 88 more columns
)

See spec(...) for full column specifications.

Warning message:
“84 parsing failures.
 row                         col           expected     actual         file
6678 fumbled_2_player_id         1/0/T/F/TRUE/FALSE 00-0033873 <connection>
6678

In [7]:
games_data

play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,⋯,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<date>,⋯,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
35,2019090500,CHI,GB,GB,away,CHI,CHI,35,2019-09-05,⋯,NA,NA,NA,0,NA,NA,0,0,0,0
50,2019090500,CHI,GB,GB,away,CHI,GB,75,2019-09-05,⋯,NA,NA,NA,0,NA,NA,0,0,0,0
71,2019090500,CHI,GB,GB,away,CHI,GB,75,2019-09-05,⋯,NA,NA,NA,0,NA,NA,0,0,0,0
95,2019090500,CHI,GB,GB,away,CHI,GB,75,2019-09-05,⋯,NA,NA,NA,0,NA,NA,0,0,0,0
125,2019090500,CHI,GB,GB,away,CHI,GB,85,2019-09-05,⋯,NA,NA,NA,0,NA,NA,0,0,0,0
155,2019090500,CHI,GB,CHI,home,GB,CHI,57,2019-09-05,⋯,00-0033120,K.Clark,5,0,NA,Defensive Holding,0,0,0,0
195,2019090500,CHI,GB,CHI,home,GB,CHI,52,2019-09-05,⋯,NA,NA,NA,0,NA,NA,0,0,0,0
216,2019090500,CHI,GB,CHI,home,GB,GB,47,2019-09-05,⋯,NA,NA,NA,0,NA,NA,0,0,0,0
238,2019090500,CHI,GB,CHI,home,GB,GB,47,2019-09-05,⋯,NA,NA,NA,0,NA,NA,0,0,0,0


In [8]:
games_data %>%
glimpse()

Rows: 45,546
Columns: 256
$ play_id                              <dbl> 35, 50, 71, 95, 125, 155, 195, 2…
$ game_id                              <dbl> 2019090500, 2019090500, 20190905…
$ home_team                            <chr> "CHI", "CHI", "CHI", "CHI", "CHI…
$ away_team                            <chr> "GB", "GB", "GB", "GB", "GB", "G…
$ posteam                              <chr> "GB", "GB", "GB", "GB", "GB", "C…
$ posteam_type                         <chr> "away", "away", "away", "away", …
$ defteam                              <chr> "CHI", "CHI", "CHI", "CHI", "CHI…
$ side_of_field                        <chr> "CHI", "GB", "GB", "GB", "GB", "…
$ yardline_100                         <dbl> 35, 75, 75, 75, 85, 57, 52, 47, …
$ game_date                            <date> 2019-09-05, 2019-09-05, 2019-09…
$ quarter_seconds_remaining            <dbl> 900, 900, 873, 825, 795, 781, 75…
$ half_seconds_remaining               <dbl> 1800, 1800, 1773, 1725, 1695, 16…
$ game_seconds_remaining  

In [10]:
pbp_rp <- games_data %>% 
	filter(play_type=="no_play" | play_type=="pass" | play_type=="run")

In [11]:
pbp_rp

play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,⋯,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<date>,⋯,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
50,2019090500,CHI,GB,GB,away,CHI,GB,75,2019-09-05,⋯,NA,NA,NA,0,NA,NA,0,0,0,0
71,2019090500,CHI,GB,GB,away,CHI,GB,75,2019-09-05,⋯,NA,NA,NA,0,NA,NA,0,0,0,0
95,2019090500,CHI,GB,GB,away,CHI,GB,75,2019-09-05,⋯,NA,NA,NA,0,NA,NA,0,0,0,0
155,2019090500,CHI,GB,CHI,home,GB,CHI,57,2019-09-05,⋯,00-0033120,K.Clark,5,0,NA,Defensive Holding,0,0,0,0
195,2019090500,CHI,GB,CHI,home,GB,CHI,52,2019-09-05,⋯,NA,NA,NA,0,NA,NA,0,0,0,0
216,2019090500,CHI,GB,CHI,home,GB,GB,47,2019-09-05,⋯,NA,NA,NA,0,NA,NA,0,0,0,0
238,2019090500,CHI,GB,CHI,home,GB,GB,47,2019-09-05,⋯,NA,NA,NA,0,NA,NA,0,0,0,0
260,2019090500,CHI,GB,CHI,home,GB,GB,40,2019-09-05,⋯,NA,NA,NA,0,NA,NA,0,0,0,0
282,2019090500,CHI,GB,CHI,home,GB,GB,40,2019-09-05,⋯,NA,NA,NA,0,NA,NA,0,0,0,0


In [15]:
pbp_rp <- pbp_rp %>%
	mutate(
	pass = if_else(str_detect(desc, "( pass)|(sacked)|(scramble)"), 1, 0),
	rush = if_else(str_detect(desc, "(left end)|(left tackle)|(left guard)|(up the middle)|(right guard)|(right tackle)|(right end)") & pass == 0, 1, 0),
	success = ifelse(epa>0, 1 , 0)
	) 

In [16]:
pbp_rp <- pbp_rp %>% filter(pass==1 | rush==1)

In [19]:
pbp_rp %>%
	filter(posteam == "SF", rush == 1, down<=4) %>%
	group_by(rusher_player_name) %>%
	summarize(mean_epa = mean(epa), success_rate = mean(success), ypc=mean(yards_gained), plays=n()) %>%
	arrange(desc(mean_epa)) %>%
	filter(plays>13)

`summarise()` ungrouping output (override with `.groups` argument)



rusher_player_name,mean_epa,success_rate,ypc,plays
<chr>,<dbl>,<dbl>,<dbl>,<int>
D.Samuel,0.85072806,0.5714286,11.357143,14
J.Wilson,0.05988834,0.4444444,3.888889,27
R.Mostert,0.04916458,0.3795620,5.635036,137
M.Breida,-0.11237335,0.3333333,5.065041,123
T.Coleman,-0.12548743,0.2773723,3.970803,137
